In [2]:
# Montar Google Drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Bibliotecas:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

In [11]:
# Construção do modelo:
def create_model():
    classifier = tf.keras.models.Sequential()

    classifier.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=3, padding="same", input_shape=(64, 64, 3),
                                                 activation='relu'))
    classifier.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    classifier.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=3, padding="same", activation="relu"))
    classifier.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    classifier.add(tf.keras.layers.Flatten())
    classifier.add(tf.keras.layers.Dense(units=128, activation='relu'))
    classifier.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    return classifier

In [12]:
# Preparação dos dados:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/CNN_COVID/Dados_divididos/train',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/CNN_COVID/Dados_divididos/test',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

Found 1459 images belonging to 2 classes.
Found 625 images belonging to 2 classes.


In [13]:
# Teste de hiperparâmetros:
log = []
num_epochs = [5, 10, 15]
batch_sizes = [16, 32, 64]
learning_rates = [0.1, 0.01, 0.005, 0.001, 0.0001]

for num_epoch in num_epochs:
    for batch_size in batch_sizes:
        for learning_rate in learning_rates:
            NAME = f"epochs-{num_epoch}-batch_size-{batch_size}-learning_rate-{learning_rate}-{int(time.time())}"
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

            classifier = create_model()
            classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                               loss='binary_crossentropy',
                               metrics=['accuracy'])

            history = classifier.fit(training_set,
                                     steps_per_epoch=4,
                                     epochs=num_epoch,
                                     validation_data=test_set,
                                     validation_steps=4,
                                     callbacks=[tensorboard])

            log_entry = {
                'num_epoch': num_epoch,
                'batch_size': batch_size,
                'learning_rate': learning_rate,
                'final_train_accuracy': history.history['accuracy'][-1],
                'final_val_accuracy': history.history['val_accuracy'][-1],
            }
            log.append(log_entry)

            print(f"Finished training with {log_entry}")

Epoch 1/5
4/4 [==============================] - 70s 21s/step - loss: 3166.9224 - accuracy: 0.5000 - val_loss: 28.0671 - val_accuracy: 0.2812
Epoch 2/5
4/4 [==============================] - 56s 17s/step - loss: 8.4101 - accuracy: 0.3828 - val_loss: 0.6015 - val_accuracy: 0.7109
Epoch 3/5
4/4 [==============================] - 51s 16s/step - loss: 0.5908 - accuracy: 0.7217 - val_loss: 0.5807 - val_accuracy: 0.7422
Epoch 4/5
4/4 [==============================] - 36s 11s/step - loss: 0.5989 - accuracy: 0.7500 - val_loss: 0.5427 - val_accuracy: 0.7734
Epoch 5/5
4/4 [==============================] - 30s 9s/step - loss: 0.5406 - accuracy: 0.7734 - val_loss: 0.5777 - val_accuracy: 0.7422
Finished training with {'num_epoch': 5, 'batch_size': 16, 'learning_rate': 0.1, 'final_train_accuracy': 0.7734375, 'final_val_accuracy': 0.7421875}
Epoch 1/5
4/4 [==============================] - 26s 8s/step - loss: 6.3817 - accuracy: 0.4922 - val_loss: 0.4387 - val_accuracy: 0.8203
Epoch 2/5
4/4 [=======

In [14]:
# Interpretando os resultados:

best_combination = max(log, key=lambda x: x['final_val_accuracy'])
print("Melhor combinação de hiperparâmetros:")
print(f"Épocas: {best_combination['num_epoch']}")
print(f"Tamanho do Lote (Batch Size): {best_combination['batch_size']}")
print(f"Taxa de Aprendizado: {best_combination['learning_rate']}")
print(f"Acurácia de Treinamento: {best_combination['final_train_accuracy']}")
print(f"Acurácia de Validação: {best_combination['final_val_accuracy']}")

sorted_log = sorted(log, key=lambda x: x['final_val_accuracy'], reverse=True)
print("Todas as combinações, ordenadas pela acurácia de validação:")
for entry in sorted_log:
    print(f"Épocas: {entry['num_epoch']}, Tamanho do Lote: {entry['batch_size']}, Taxa de Aprendizado: {entry['learning_rate']}, Acurácia de Treinamento: {entry['final_train_accuracy']:.2f}, Acurácia de Validação: {entry['final_val_accuracy']:.2f}")

Melhor combinação de hiperparâmetros:
Épocas: 15
Tamanho do Lote (Batch Size): 16
Taxa de Aprendizado: 0.001
Acurácia de Treinamento: 0.921875
Acurácia de Validação: 0.96875
Todas as combinações, ordenadas pela acurácia de validação:
Épocas: 15, Tamanho do Lote: 16, Taxa de Aprendizado: 0.001, Acurácia de Treinamento: 0.92, Acurácia de Validação: 0.97
Épocas: 15, Tamanho do Lote: 32, Taxa de Aprendizado: 0.001, Acurácia de Treinamento: 0.98, Acurácia de Validação: 0.97
Épocas: 15, Tamanho do Lote: 64, Taxa de Aprendizado: 0.005, Acurácia de Treinamento: 0.95, Acurácia de Validação: 0.97
Épocas: 10, Tamanho do Lote: 32, Taxa de Aprendizado: 0.01, Acurácia de Treinamento: 0.90, Acurácia de Validação: 0.96
Épocas: 10, Tamanho do Lote: 32, Taxa de Aprendizado: 0.001, Acurácia de Treinamento: 0.94, Acurácia de Validação: 0.96
Épocas: 5, Tamanho do Lote: 32, Taxa de Aprendizado: 0.001, Acurácia de Treinamento: 0.90, Acurácia de Validação: 0.95
Épocas: 10, Tamanho do Lote: 16, Taxa de Aprendi